In [1]:
import os
os.chdir('../')
%pwd

'/home/dheeraj37/Projects/Audio-Genre-Classification-1'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareDataConfig:
    root_dir: Path
    source_dir: Path
    target_dir: Path

In [3]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_prepare_data_config(self) -> PrepareDataConfig:
        config = self.config.prepare_data

        create_directories([config.root_dir])

        prepare_data_config = PrepareDataConfig(
            root_dir=config.root_dir,
            source_dir=config.source_dir,
            target_dir=config.target_dir,
        )

        return prepare_data_config

In [35]:
import os
import pandas as pd
import librosa
import numpy as np
import urllib.request as request
import zipfile
from cnnClassifier import logger

In [88]:
class PrepareData:
    def __init__(self, config: PrepareDataConfig):
        self.config = config       
        
    def dataframe_create(self):
        self.music_dataset = [] # File locations for each .wav file 
        self.genre_list = [] # Different Genres (or classes)
        self.music_path = self.config.source_dir
        
        for root, dirs, files in os.walk(self.music_path):
            for name in files:
                filename = os.path.join(root, name)
                self.music_dataset.append(filename)
                self.genre_list.append(filename.split("/")[3])
            self.music_df = pd.DataFrame({
            'file': self.music_dataset,
            'genre': self.genre_list
            })
        return self.music_df   
    
    def mfcc_feature_extraction(self, df):
        
        self.X = []
        self.y = []
        self.y_cat = []
        self.mfcc = []
        self.mfcc_df = pd.DataFrame()

        for index, row in df.iterrows():
            signal, sr = librosa.load(row['file'], sr=22050)
            if len(signal) > 661500:
                signal = signal[0:661500]
            else:
                signal = np.pad(signal, (0, 661500 - signal.shape[0]), 'wrap')
                
            # Calculate the duration of each chunk
            chunk_duration = 5  # seconds
            # Calculate the number of samples in each chunk
            chunk_samples = int(chunk_duration * sr)
            # Calculate the total number of chunks
            num_chunks = int(np.ceil(len(signal) / chunk_samples))
            # Split the audio clip into chunks
            num_chunks = int(np.ceil(len(signal) / chunk_samples))
            audio_chunks = [signal[i*chunk_samples:(i+1)*chunk_samples] for i in range(num_chunks)]
            for chunk in audio_chunks:
                mfcc = librosa.feature.mfcc(y=chunk, n_fft=2048, hop_length=512, n_mfcc=13, sr=sr)
                mfcc = np.array(mfcc.T) # Transpose the matrix to get the shape (n, 13)
                
                self.X.append(mfcc)
                self.y.append(row['genre'])

        self.mfcc_df['MFCC features'] = self.X
        self.mfcc_df['genre'] = self.y
        
        return self.mfcc_df    

    def save_data(self, df):
        df.to_json(self.config.target_dir, index=False)
        logger.info(f"Data saved at {self.config.target_dir} in JSON format")          
        



In [89]:
try:
    config = ConfigurationManager()
    prepare_data_config = config.get_prepare_data_config()
    prepare_data = PrepareData(config=prepare_data_config)
    music_df = prepare_data.dataframe_create()
    mfcc_df = prepare_data.mfcc_feature_extraction(music_df)
    prepare_data.save_data(mfcc_df)

except Exception as e:
    raise e

[2023-09-23 19:59:51,848]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-23 19:59:51,856]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-23 19:59:51,857]: INFO: common: created directory at: artifacts
[2023-09-23 19:59:51,857]: INFO: common: created directory at: artifacts/prepare_data
[2023-09-23 20:00:55,066]: INFO: 228012998: Data saved at artifacts/prepare_data/mfcc_5sec_chunks.json in JSON format


In [90]:
mfcc_df = pd.read_json("artifacts/prepare_data/mfcc_5sec_chunks.json")